In [1]:
def printMatrix(mat):
    for i in range(len(mat)):
        for j in range(len(mat[i])):
            print(mat[i][j], end=" ")
        print()

def constructMatrix(str1, str2):
    list = [[0 for i in range(len(str1)+1)] for j in range(len(str2)+1)]
    return list

def printStringMatrix(matrix, str1, str2):
    print("", end = "  ")
    print("_", end = " ")
    for i in range(len(str1)):
        print(str1[i], end = " ")
    print()
    for i in range(len(matrix)):
        if i == 0 :
            print("_", end = " ")
        else:
            print(str2[i-1], end = " ")
        for j in range(len(matrix[i])):
            print(matrix[i][j], end = " ")
        print()

def initMatrix(matrix, gapPenalty):
    x=0
    for i in range(len(matrix)):
        matrix[i][0] = x * gapPenalty
        x=x+1
    x=0
    for i in range(len(matrix[0])):
        matrix[0][i] = x * gapPenalty
        x=x+1
    return matrix

def getMinPenalty(matrix, i, j, str1, str2,matchPenalty, mismatchPenalty, gapPenalty ):
    if str2[i-1] == str1[j-1]:
        currPenalty = matchPenalty
    else:
        currPenalty = mismatchPenalty

    top = matrix[i-1][j] + gapPenalty
    left = matrix[i][j-1] + gapPenalty
    diag = matrix[i-1][j-1] + currPenalty
    maxE = max(top, left, diag)
    currPenalty = maxE

    return currPenalty

def fillMatrix(matrix, str1, str2, matchPenalty, mismatchPenalty, gapPenalty):
    for i in range(1, len(matrix)):
        for j in range(1, len(matrix[0])):
            matrix[i][j] = getMinPenalty(matrix, i, j, str1, str2, matchPenalty, mismatchPenalty, gapPenalty)


def getPrevious(matrix, i, j,matchPenalty, mismatchPenalty, gapPenalty ):

    score = matrix[i][j]
    top = matrix[i-1][j]
    left = matrix[i][j-1]
    diag = matrix[i-1][j-1]

    maxPrevIndexI = 0
    maxPrevIndexJ = 0
    maxPrevDir = 0
    if diag + matchPenalty == score or diag + mismatchPenalty == score:
        maxPrevIndexI = i-1
        maxPrevIndexJ = j-1
        maxPrevDir = 0
    elif top + gapPenalty == score:
        maxPrevIndexI = i-1
        maxPrevIndexJ = j
        maxPrevDir = 1
    elif left + gapPenalty == score:
        maxPrevIndexI = i
        maxPrevIndexJ = j-1
        maxPrevDir = 2

    return maxPrevDir, maxPrevIndexI, maxPrevIndexJ

def score(matrix):
    return matrix[-1][-1]

def backTrack(matrix, matchPenalty, mismatchPenalty, gapPenalty):

    score = 0
    directions = []

    i = len(matrix)-1
    j = len(matrix[0])-1

    while i>=0 and j>=0:
        score += matrix[i][j]
        if i == 0 or j == 0:
            break
        prev = getPrevious(matrix, i, j, matchPenalty, mismatchPenalty, gapPenalty)
        directions.insert(0, prev[0])
        i = prev[1]
        j = prev[2]
    return directions, score


In [2]:
def alignSequences(sequence1, sequence2, matchPenalty = 1, mismatchPenalty = -1, gapPenalty = -2):
    mat = constructMatrix(sequence1, sequence2)
    matrix = initMatrix(mat, gapPenalty)
    fillMatrix(matrix, sequence1, sequence2, matchPenalty, mismatchPenalty, gapPenalty)
    return matrix[-1][-1]
    # dir = backTrack(matrix, matchPenalty, mismatchPenalty, gapPenalty)
    # return dir[1]

In [3]:
alignSequences("GGGGGCCATCGTAATGTCACCTTT", "AAGGGCCATAGGTGTGTTAACCTC")

4

In [4]:
from Bio import SeqIO
sequences = {}
for record in SeqIO.parse("sequences_24.fasta", "fasta"):
    sequences[record.id] = str(record.seq)
print(len(sequences))

529


In [5]:
sequences

{'C001': 'GGGAGCCATCGTAATGTTTCCCCC',
 'C002': 'GGGGGCCACCGTAATGCCTCCCCC',
 'C003': 'GAGAGCCATAGGAGTGCCAACCTT',
 'C004': 'GAGGGCCATAGGAGAACTACTCTC',
 'C005': 'AGGGGCCATAGTAAAGCCAACTTC',
 'C006': 'GGGAGCGGTATTAATGTTACCTTT',
 'C007': 'GGGAGCGGTATTAATGTTACCTTT',
 'C008': 'GGGGGCGGTATGTATGTTACCTTC',
 'C009': 'GGGAGCCGTATGAATGTTACCTTT',
 'C010': 'GAGGGCCGTATGTATGCCAACCTT',
 'C011': 'GGGAGCGGTATGTATGTTACCTTT',
 'C012': 'GAAAGCCATAGGTGAGTTAACCTT',
 'C013': 'GGGGGCCACCGTAATGTCTCCCTC',
 'C014': 'GAAAGCGGTATGTGTACCAACTTT',
 'C015': 'GGGGGCCACAGTAATGTCTCCCCC',
 'C016': 'GAGAGCCATAGGTGTACCAACCCT',
 'C017': 'GAAGGCCATAGGTGTACCAACCTT',
 'C018': 'GAAAGCCATAGGTGTACCAACCTT',
 'C019': 'GGGAGCGGTATTAATGTTACCTTT',
 'C020': 'GGGAGCGGTAGTAATGTTACCTCT',
 'C021': 'GGGAGCGGTATTAATGCTACCTCT',
 'C022': 'GGGAGCGATCGGTATGTTACCTTT',
 'C023': 'GAGGGCCATAGGTGTACCAACCTT',
 'C024': 'GGAAGCGATCGTAATGTTAACTCT',
 'C025': 'GGGGGCGGTATGTATGTTACCTTT',
 'C026': 'GAAAGCCATAGGTGTACCAACCTT',
 'C027': 'GGGAGCGGTATGAATGTTACCTTC',
 

In [6]:
# NNGGGNCANNNNNNNNNCNNNCTC
# GAGGGCCATAGGTATGTCACCTTT
# GAAAGCCATAGGNNTGCNAACNNT
# GGGGGCCACCGTAATGCCTCCCCC
# AA

In [7]:
querySequence = "GGGAGCGGTAGGTATGTTACCTTT"
alignmentScore = dict()
for k,v in sequences.items():
    alignmentScore[k] = alignSequences(querySequence, v)
print(alignmentScore)

{'C001': 6, 'C002': -2, 'C003': 6, 'C004': 0, 'C005': 2, 'C006': 18, 'C007': 18, 'C008': 18, 'C009': 18, 'C010': 8, 'C011': 22, 'C012': 8, 'C013': 2, 'C014': 8, 'C015': 2, 'C016': 4, 'C017': 2, 'C018': 4, 'C019': 18, 'C020': 18, 'C021': 14, 'C022': 20, 'C023': 4, 'C024': 10, 'C025': 20, 'C026': 4, 'C027': 18, 'C028': 4, 'C029': 8, 'C030': 18, 'C031': 18, 'C032': 2, 'C033': 12, 'C034': 10, 'C035': 0, 'C036': 10, 'C037': 18, 'C038': 11, 'C039': 14, 'C040': 20, 'C041': 18, 'C042': 18, 'C043': 20, 'C044': 16, 'C045': 18, 'C046': 20, 'C047': 10, 'C048': 10, 'C049': 18, 'C050': 20, 'C051': 12, 'C052': 0, 'C053': 20, 'C054': 10, 'C055': 14, 'C056': 4, 'C057': 2, 'C058': 16, 'C059': 16, 'C060': 12, 'C061': 22, 'C062': 22, 'C063': -1, 'C064': 10, 'C065': 24, 'C066': 14, 'C067': 6, 'C068': 20, 'C069': 22, 'C070': 5, 'C071': 8, 'C072': 12, 'C073': 10, 'C074': -2, 'C075': 6, 'C076': 10, 'C077': 8, 'C078': 8, 'C079': 6, 'C080': 6, 'C081': 10, 'C082': 6, 'C083': 10, 'C084': 4, 'C085': 2, 'C086': 4, 

In [8]:
sorted_dict_by_value_desc = dict(sorted(alignmentScore.items(), key=lambda item: item[1], reverse=True))
sorted_dict_by_value_desc

{'C065': 24,
 'W161': 24,
 'W202': 24,
 'C011': 22,
 'C061': 22,
 'C062': 22,
 'C069': 22,
 'C168': 22,
 'W051': 22,
 'W166': 22,
 'W167': 22,
 'W168': 22,
 'W176': 22,
 'W177': 22,
 'W182': 22,
 'W184': 22,
 'W195': 22,
 'W211': 22,
 'W213': 22,
 'W216': 22,
 'W227': 22,
 'W242': 22,
 'W262': 22,
 'W301': 22,
 'W311': 22,
 'W316': 22,
 'C022': 20,
 'C025': 20,
 'C040': 20,
 'C043': 20,
 'C046': 20,
 'C050': 20,
 'C053': 20,
 'C068': 20,
 'C160': 20,
 'C170': 20,
 'C178': 20,
 'W085': 20,
 'W086': 20,
 'W146': 20,
 'W147': 20,
 'W150': 20,
 'W153': 20,
 'W163': 20,
 'W171': 20,
 'W186': 20,
 'W197': 20,
 'W201': 20,
 'W231': 20,
 'W234': 20,
 'W238': 20,
 'W239': 20,
 'W267': 20,
 'W268': 20,
 'W289': 20,
 'W293': 20,
 'W299': 20,
 'W321': 20,
 'C006': 18,
 'C007': 18,
 'C008': 18,
 'C009': 18,
 'C019': 18,
 'C020': 18,
 'C027': 18,
 'C030': 18,
 'C031': 18,
 'C037': 18,
 'C041': 18,
 'C042': 18,
 'C045': 18,
 'C049': 18,
 'C107': 18,
 'C142': 18,
 'C147': 18,
 'C148': 18,
 'C151': 18,

In [9]:
k = 3

In [10]:
topk = list(sorted_dict_by_value_desc.keys())[:k]
topk

['C065', 'W161', 'W202']

In [11]:
import pandas as pd
df = pd.read_csv("mapping_24.csv")
df['Subpopulation']= df['Subpopulation'].astype("string")

In [12]:
print(df.columns)

Index(['Cultivar ID', 'vg0112116426', 'vg0128525986', 'vg0130976864',
       'vg0131664768', 'vg0133440209', 'vg0135617816', 'vg0135642980',
       'vg0138418739', 'vg0138428840', 'vg0138608956', 'vg0138999212',
       'vg0405463422', 'vg0405463763', 'vg0603483061', 'vg0713178880',
       'vg0719727299', 'vg0719727339', 'vg0719834473', 'vg0819793460',
       'vg0904094998', 'vg0904282939', 'vg1019044175', 'vg1123563633',
       'vg1207667840', 'Subpopulation', 'Plant Height (cm)', 'sequence'],
      dtype='object')


In [13]:
for cultivar in topk:
    row = df.loc[df['Cultivar ID'] == cultivar]
    print(cultivar, row['Subpopulation'].item() )

C065 Indica Intermediate
W161 Indica II
W202 Indica Intermediate


In [14]:
import json

# Read the JSON data from a file
with open("json/clusters_dict.json", "r") as f:
    clusters_dict = json.loads(f.read())

with open("json/sequence_cluster_dict.json", "r") as f:
    sequence_cluster_dict = json.loads(f.read())

with open("json/sequences_dict.json", "r") as f:
    sequences_dict = json.loads(f.read())

with open("json/unique_sequences_per_cluster.json", "r") as f:
    unique_sequences_per_cluster = json.loads(f.read())


In [15]:
clusters = []
subpops = []
for cultivar in topk:
    row = df.loc[df['Cultivar ID'] == cultivar]
    print(cultivar, row['Subpopulation'].item(), sequence_cluster_dict[cultivar])
    clusters.append(sequence_cluster_dict[cultivar])
    subpops.append(row['Subpopulation'].item())

C065 Indica Intermediate 92
W161 Indica II 92
W202 Indica Intermediate 92


In [16]:
cluster_belong = sorted(clusters, key= lambda x : clusters.count(x), reverse=True)[0]
cluster_belong

92

In [17]:
subpop_belong = sorted(subpops, key = lambda x : subpops.count(x), reverse=True)[0]
subpop_belong

'Indica Intermediate'

In [18]:
df_new = df[df['Cultivar ID'].isin(clusters_dict[str(cluster_belong)])]
df_new = df_new[df_new['Subpopulation'] == subpop_belong]
df_new

,Cultivar ID,vg0112116426,vg0128525986,vg0130976864,vg0131664768,vg0133440209,vg0135617816,vg0135642980,vg0138418739,vg0138428840,...,vg0719834473,vg0819793460,vg0904094998,vg0904282939,vg1019044175,vg1123563633,vg1207667840,Subpopulation,Plant Height (cm),sequence
64,C065,G,G,G,A,G,C,G,G,T,...,T,A,C,C,T,T,T,Indica Intermediate,109.07,GGGAGCGGTAGGTATGTTACCTTT
401,W202,G,G,G,A,G,C,G,G,T,...,T,A,C,C,T,T,T,Indica Intermediate,112.00,GGGAGCGGTAGGTATGTTACCTTT


In [19]:
heights = df_new['Plant Height (cm)']
print("Height range : ", heights.min(), " - ", heights.max())

Height range :  109.07  -  112.0


In [38]:
def height_range(query_sequence, nk=5):

    # finding alignment scores
    alignmentScore = dict()
    for k,v in sequences.items():
        alignmentScore[k] = alignSequences(query_sequence, v)

    # sorting based on highest scored, and considering top k matches
    sorted_dict_by_value_desc = dict(sorted(alignmentScore.items(), key=lambda item: item[1], reverse=True))
    topk = list(sorted_dict_by_value_desc.keys())[:nk]
    clusters = []
    subpops = []
    for cultivar in topk:
        row = df.loc[df['Cultivar ID'] == cultivar]
        clusters.append(sequence_cluster_dict[cultivar])
        subpops.append(row['Subpopulation'].item())

    # finding range of height
    cluster_belong = sorted(clusters, key= lambda x : clusters.count(x), reverse=True)[0]
    subpop_belong = sorted(subpops, key = lambda x : subpops.count(x), reverse=True)[0]
    df_new = df[df['Cultivar ID'].isin(clusters_dict[str(cluster_belong)])]
    df_new = df_new[df_new['Subpopulation'] == subpop_belong]
    heights = df_new['Plant Height (cm)']


    # print("Height range : ", heights.min(), " - ", heights.min(),)
    return heights.min(),heights.max()

In [21]:
height_range("GGGAGCGGTATTAATGTTACCTTT", nk=3)

(91.4, 118.07)

In [29]:
# W062, W123, W218, C034, C054
#
# C179, W062, W123, C034, C054

In [30]:
# 97.9  -  110.5

In [31]:
# ML model to decide weights, highest match vs the majority of top k matches

In [22]:
validation = pd.read_csv('validation.csv').iloc[:,1:]
validation

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,sequence
0,G,A,G,N,G,C,C,A,T,N,...,C,A,N,C,NaN,NaN,NaN,TEMPORATE JAPONICA,126.0,GAGNGCCATNGGTNNGCCANC
1,N,N,N,C,C,N,N,C,G,N,...,N,C,T,T,NaN,NaN,NaN,INDICA,170.0,NNNCCNNCGNNNNNNCANCTT
2,G,N,G,N,N,N,C,N,N,N,...,C,N,N,C,N,T,N,AUS,110.0,GNGNNNCNNNNNNNNNNCNNCNTN
3,N,G,G,G,G,C,C,N,T,A,...,T,A,A,C,C,N,C,AROMATIC,130.0,NGGGGCCNTAGNNGNGTTAACCNC
4,G,A,G,N,G,C,C,A,T,A,...,C,A,A,C,C,T,T,TROPICAL,169.0,GAGNGCCATAGGTNAGCCAACCTT
5,N,N,A,A,G,C,N,G,T,A,...,N,N,N,C,T,NaN,NaN,TEMPORATE JAPONICA,97.9,NNAAGCNGTATGTNTNNNNNCT
6,G,N,G,N,N,N,N,G,T,A,...,N,A,N,NaN,NaN,NaN,NaN,INDICA,106.0,GNGNNNNGTATNNNTNNNAN
7,N,N,A,N,G,C,N,A,T,C,...,N,A,A,C,N,NaN,NaN,AUS,137.0,NNANGCNATCGNNNTNNNAACN
8,N,N,G,N,N,N,C,A,N,C,...,C,N,N,C,C,NaN,NaN,intermediate,137.0,NNGNNNCANCGNNATNNCNNCC
9,G,G,A,C,G,G,N,A,T,G,...,N,C,N,C,T,NaN,NaN,indica,96.0,GGACGGNATGTNNNNNANCNCT


In [23]:
for index, data in validation.iterrows():
    print(index, data['sequence'], data[25])

0 GAGNGCCATNGGTNNGCCANC 126.0
1 NNNCCNNCGNNNNNNCANCTT 170.0
2 GNGNNNCNNNNNNNNNNCNNCNTN 110.0
3 NGGGGCCNTAGNNGNGTTAACCNC 130.0
4 GAGNGCCATAGGTNAGCCAACCTT 169.0
5 NNAAGCNGTATGTNTNNNNNCT 97.9
6 GNGNNNNGTATNNNTNNNAN 106.0
7 NNANGCNATCGNNNTNNNAACN 137.0
8 NNGNNNCANCGNNATNNCNNCC 137.0
9 GGACGGNATGTNNNNNANCNCT 96.0
10 AGAGCCATAGNNGNGNNAACCT 128.0
11 GAAAGCCATAGGTGTACCAACCTT 147.0
12 GAAAGCCATAGG 98.0
13 GAGGGCCATAGGTATGTCACCTTT 177.0


In [24]:
header = ['k', 'sequence', 'min', 'max', 'height', 'inRange']
test = pd.DataFrame(columns = header)
k_values = [1,3, 5, 7, 9]
for k in k_values:
    for row_index, row_data in validation.iterrows():

        # Access values of each row
        sequence = row_data['sequence']
        l = [k, sequence]
        min_h, max_h = height_range(sequence, nk=k)
        height = row_data[25]

        isInRange = False
        if not (min_h=="NaN" or max_h=="NaN"):
            if max_h +10 >= height >= min_h-10:
                isInRange = True

        l.append(min_h)
        l.append(max_h)
        l.append(height)
        l.append(isInRange)
        row_df = pd.DataFrame([l], columns=header)
        test = pd.concat([test, row_df], ignore_index=True)
test

,k,sequence,min,max,height,inRange
0,1,GAGNGCCATNGGTNNGCCANC,105.07,146.00,126.0,True
1,1,NNNCCNNCGNNNNNNCANCTT,170.17,170.17,170.0,True
2,1,GNGNNNCNNNNNNNNNNCNNCNTN,141.57,141.57,110.0,False
3,1,NGGGGCCNTAGNNGNGTTAACCNC,174.80,174.80,130.0,False
4,1,GAGNGCCATAGGTNAGCCAACCTT,153.40,169.40,169.0,True
...,...,...,...,...,...,...
65,9,GGACGGNATGTNNNNNANCNCT,95.60,123.80,96.0,True
66,9,AGAGCCATAGNNGNGNNAACCT,117.93,131.30,128.0,True
67,9,GAAAGCCATAGGTGTACCAACCTT,68.40,107.00,147.0,False
68,9,GAAAGCCATAGG,68.40,107.00,98.0,True


In [25]:
height_range("GNGNNNCNNNNNNNNNNCNNCNTN", nk=5)

In [26]:
test.to_csv("results.csv", index=None)

{1: 8, 3: 9, 5: 7, 7: 6, 9: 7}


In [39]:
accuracy = dict()
for index, row in test.iterrows():
        if row['inRange']:
            kn = row['k']
            if kn in accuracy:
                accuracy[kn] +=1
            else:
                accuracy[kn] = 1
print(accuracy)

[40, 2, 145, 13, 25] ['Japonica Intermediate', 'Tropical Japonica', 'Aus', 'Temperate Japonica', 'Temperate Japonica']


(nan, nan)